# 读取数据

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pymongo
import pandas as pd
from pandas import Series
client = pymongo.MongoClient('localhost',27017)
db  = client['Graduation_project']
table = db['shanghai_discovery']
data = pd.DataFrame(list(table.find()))
del data['_id']

In [3]:
data.head()

,Keyword,公司名称,公司规模,地区,学历要求,工作经验,职位名称,职位描述,职位薪资,融资情况
0,数据分析,立翱,20-99人,上海 杨浦区 五角场,本科,1-3年,数据分析专员,岗位职责：1、 负责公司的数据分析、数据挖掘、分析报告、 按时、准确的提供各指标的日数据报表...,6k-7k,未融资
1,数据分析,上海恒奕集团,500-999人,上海 普陀区 武宁路,大专,1-3年,数据分析,岗位职责：通过抓取第三方数据来分析在不同地区，我们医院项目的目标人群画像分析，实时网络热度盘...,8k-12k,A轮
2,数据分析,晶赞科技,100-499人,上海 静安区 共和新路,本科,3-5年,数据分析,工作职责：1、负责政务数据/商业数据的研究和分析。例如，通过政务公开数据的挖掘和分析，形成主...,12k-24k,D轮及以上
3,数据分析,东方购物,1000-9999人,上海 杨浦区 复旦大学,本科,1-3年,数据分析,工作职责：1、负责公司所有业务线数据分析；2、负责网站流量、用户行为特征分析；3、通过数据挖...,7k-10k,已上市
4,数据分析,网达软件,1000-9999人,上海 杨浦区 长阳路,大专,1-3年,数据分析,岗位职责：1、处理并监控业务发展相关数据，为运营、发展决策提供优化建议；2、构建并优化数据指...,6k-8k,已上市


In [4]:
data.shape

(5062, 10)

# 数据清洗

## 去重去空

In [5]:
data = data.drop_duplicates()

In [6]:
data.shape

(1905, 10)

In [7]:
data = data.drop(data[data['职位描述']==''].index)

In [8]:
data.shape

(1723, 10)

---

# 划分不同方向的岗位

In [9]:
data.Keyword.unique()

array(['数据分析', '数据挖掘', '商业分析', '机器学习'], dtype=object)

In [10]:
data_analysis = data[data.Keyword=='数据分析']
data_mining = data[data.Keyword=='数据挖掘']
business_analysis = data[data.Keyword=='商业分析']
machine_learning = data[data.Keyword=='机器学习']

In [11]:
data_analysis.head()

,Keyword,公司名称,公司规模,地区,学历要求,工作经验,职位名称,职位描述,职位薪资,融资情况
0,数据分析,立翱,20-99人,上海 杨浦区 五角场,本科,1-3年,数据分析专员,岗位职责：1、 负责公司的数据分析、数据挖掘、分析报告、 按时、准确的提供各指标的日数据报表...,6k-7k,未融资
1,数据分析,上海恒奕集团,500-999人,上海 普陀区 武宁路,大专,1-3年,数据分析,岗位职责：通过抓取第三方数据来分析在不同地区，我们医院项目的目标人群画像分析，实时网络热度盘...,8k-12k,A轮
2,数据分析,晶赞科技,100-499人,上海 静安区 共和新路,本科,3-5年,数据分析,工作职责：1、负责政务数据/商业数据的研究和分析。例如，通过政务公开数据的挖掘和分析，形成主...,12k-24k,D轮及以上
3,数据分析,东方购物,1000-9999人,上海 杨浦区 复旦大学,本科,1-3年,数据分析,工作职责：1、负责公司所有业务线数据分析；2、负责网站流量、用户行为特征分析；3、通过数据挖...,7k-10k,已上市
4,数据分析,网达软件,1000-9999人,上海 杨浦区 长阳路,大专,1-3年,数据分析,岗位职责：1、处理并监控业务发展相关数据，为运营、发展决策提供优化建议；2、构建并优化数据指...,6k-8k,已上市


## 滤除不相关的职位

### 数据分析方向

In [12]:
dt_test = data_analysis.astype(str)
y = dt_test[
    dt_test['职位名称'].str.contains('讲师|实习|导师|专员')
           ]

test1 = list(y['职位名称']) # 将含有关键字的列表表示为test1
test2 = list(dt_test['职位名称']) # 将全部表示为test2
ret = list(set(test2) ^ set(test1)) # 列表求差集的方法将含有关键字的行除去

data_analysis = dt_test[dt_test['职位名称'].isin(ret)]

In [13]:
data_analysis.shape

(585, 10)

### 数据挖掘方向

In [14]:
dt_test = data_mining.astype(str)
y = dt_test[
    dt_test['职位名称'].str.contains('讲师|实习|导师|专员')
           ]

test1 = list(y['职位名称']) # 将含有关键字的列表表示为test1
test2 = list(dt_test['职位名称']) # 将全部表示为test2
ret = list(set(test2) ^ set(test1)) # 列表求差集的方法将含有关键字的行除去

data_mining = dt_test[dt_test['职位名称'].isin(ret)]

In [15]:
data_mining.shape

(411, 10)

### 商业分析方向

In [16]:
dt_test = business_analysis.astype(str)
y = dt_test[
    dt_test['职位名称'].str.contains('讲师|实习|导师|专员')
           ]

test1 = list(y['职位名称']) # 将含有关键字的列表表示为test1
test2 = list(dt_test['职位名称']) # 将全部表示为test2
ret = list(set(test2) ^ set(test1)) # 列表求差集的方法将含有关键字的行除去

business_analysis = dt_test[dt_test['职位名称'].isin(ret)]

In [17]:
business_analysis.shape

(321, 10)

### 机器学习方向

In [18]:
dt_test = machine_learning.astype(str)
y = dt_test[
    dt_test['职位名称'].str.contains('讲师|实习|导师|专员')
           ]

test1 = list(y['职位名称']) # 将含有关键字的列表表示为test1
test2 = list(dt_test['职位名称']) # 将全部表示为test2
ret = list(set(test2) ^ set(test1)) # 列表求差集的方法将含有关键字的行除去

machine_learning = dt_test[dt_test['职位名称'].isin(ret)]

In [19]:
machine_learning.shape

(296, 10)

---

## 输出到csv

In [20]:
data_analysis.to_csv('./data/data_analysis.csv')
data_mining.to_csv('./data/data_mining.csv')
business_analysis.to_csv('./data/business_analysis.csv')
machine_learning.to_csv('./data/machine_learning.csv')

---

In [21]:
data_analysis.head()

,Keyword,公司名称,公司规模,地区,学历要求,工作经验,职位名称,职位描述,职位薪资,融资情况
1,数据分析,上海恒奕集团,500-999人,上海 普陀区 武宁路,大专,1-3年,数据分析,岗位职责：通过抓取第三方数据来分析在不同地区，我们医院项目的目标人群画像分析，实时网络热度盘...,8k-12k,A轮
2,数据分析,晶赞科技,100-499人,上海 静安区 共和新路,本科,3-5年,数据分析,工作职责：1、负责政务数据/商业数据的研究和分析。例如，通过政务公开数据的挖掘和分析，形成主...,12k-24k,D轮及以上
3,数据分析,东方购物,1000-9999人,上海 杨浦区 复旦大学,本科,1-3年,数据分析,工作职责：1、负责公司所有业务线数据分析；2、负责网站流量、用户行为特征分析；3、通过数据挖...,7k-10k,已上市
4,数据分析,网达软件,1000-9999人,上海 杨浦区 长阳路,大专,1-3年,数据分析,岗位职责：1、处理并监控业务发展相关数据，为运营、发展决策提供优化建议；2、构建并优化数据指...,6k-8k,已上市
5,数据分析,PACTERA,10000人以上,上海 浦东新区 唐镇,大专,3-5年,Python数据分析,熟练使用python语言，能够熟练编写python脚本对机器学习相关技术与算法有一定了解有过...,10k-14k,已上市


# Description NLP

In [22]:
data_analysis['职位描述'][1]

'岗位职责：通过抓取第三方数据来分析在不同地区，我们医院项目的目标人群画像分析，实时网络热度盘点，各平台流量数据分析上班时间：做六休一吗，早九晚五福利待遇：包吃住，五险一金'

## 读取用户字典

In [23]:
import jieba
# 读取用户字典
jieba.load_userdict("./dict/userdict.txt")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\13626\AppData\Local\Temp\jieba.cache
Loading model cost 0.937 seconds.
Prefix dict has been built succesfully.


In [24]:
# 读取停用词
with open('./dict/Stopword.txt',"r",encoding='utf-8') as f:
    stopwords = f.read()
stopwords = stopwords.split('\n')

In [25]:
# 读取软技能字典
with open('./dict/软技能.txt',"r") as f:
    soft_skills = f.read()
skilllist = soft_skills.split('\n')

In [26]:
# # 读取硬技能字典
with open('./dict/硬技能.txt',"r") as f:
    hard_skills = f.read()

In [27]:
# 软技能匹配函数
def match_soft(skills,templist):
    for item in skills:
        if item not in stopwords:
            if item in skilllist:
                #print(item)
                templist.append(item)

In [28]:
# 硬技能匹配函数
def match_hard(skill_list,templist):
    for skill in skill_list:
        # 单词转换为首字母大写，其余小写
        if skill.capitalize() not in stopwords:
            if skill.capitalize() in hard_skills:
                templist.append(skill.capitalize())

---

## 正则提取英文单词

In [29]:
import re
pattern = '[a-zA-Z]+'

### data analysis

In [30]:
data_analysis_soft = []
data_analysis_hard = []
# 匹配软技能
for item in data_analysis['职位描述']:
    skill_list = jieba.lcut(item)
    skill_list = list(set(skill_list)) # list去重
    match_soft(skill_list,data_analysis_soft)
# 匹配硬技能
for item in data_analysis['职位描述']:
    skill_list = re.findall(pattern,item)
    skill_list = list(set(skill_list)) # list去重
    match_hard(skill_list,data_analysis_hard)

In [31]:
data_analysis_soft_counts = Series(data_analysis_soft).value_counts()[:10]
data_analysis_hard_counts = Series(data_analysis_hard).value_counts()[:10]

### data mining

In [32]:
data_mining_soft = []
data_mining_hard = []

for item in data_mining['职位描述']:
    skill_list = jieba.lcut(item)
    skill_list = list(set(skill_list)) # list去重
    match_soft(skill_list,data_mining_soft)
    
# 匹配硬技能
for item in data_mining['职位描述']:
    skill_list = re.findall(pattern,item)
    skill_list = list(set(skill_list)) # list去重
    match_hard(skill_list,data_mining_hard)

In [33]:
data_mining_soft_counts = Series(data_mining_soft).value_counts()[:10]
data_mining_hard_counts = Series(data_mining_hard).value_counts()[:10]

### Business Analysis

In [34]:
business_analysis_soft = []
business_analysis_hard = []

for item in business_analysis['职位描述']:
    skill_list = jieba.lcut(item)
    skill_list = list(set(skill_list)) # list去重
    match_soft(skill_list,business_analysis_soft)

# 匹配硬技能
for item in business_analysis['职位描述']:
    skill_list = re.findall(pattern,item)
    skill_list = list(set(skill_list)) # list去重
    match_hard(skill_list,business_analysis_hard)

In [35]:
business_analysis_soft_counts = Series(business_analysis_soft).value_counts()[:10]
business_analysis_hard_counts = Series(business_analysis_hard).value_counts()[:10]

### Machine Learning

In [36]:
machine_learning_soft = []
machine_learning_hard = []

for item in machine_learning['职位描述']:
    skill_list = jieba.lcut(item)
    skill_list = list(set(skill_list)) # list去重
    match_soft(skill_list,machine_learning_soft)
    
# 匹配硬技能
for item in machine_learning['职位描述']:
    skill_list = re.findall(pattern,item)
    skill_list = list(set(skill_list)) # list去重
    match_hard(skill_list,machine_learning_hard)

In [37]:
machine_learning_soft_counts = Series(machine_learning_soft).value_counts()[:10]
machine_learning_hard_counts = Series(machine_learning_hard).value_counts()[:10]

## Frame

### SOFT

In [38]:
from pandas import DataFrame

temp_data = {
    '数据分析':data_analysis_soft_counts.index,
    'count1':data_analysis_soft_counts.values,
    '数据挖掘':data_mining_soft_counts.index,
    'count2':data_mining_soft_counts.values,
    '商业分析':business_analysis_soft_counts.index,
    'count3':business_analysis_soft_counts.values,
    '机器学习':machine_learning_soft_counts.index,
    'count4':machine_learning_soft_counts.values,
}
frame_soft = DataFrame(temp_data,index=[1,2,3,4,5,6,7,8,9,10])

### HARD

In [39]:
temp_data = {
    '数据分析':data_analysis_hard_counts.index,
    'count1':data_analysis_hard_counts.values,
    '数据挖掘':data_mining_hard_counts.index,
    'count2':data_mining_hard_counts.values,
    '商业分析':business_analysis_hard_counts.index,
    'count3':business_analysis_hard_counts.values,
    '机器学习':machine_learning_hard_counts.index,
    'count4':machine_learning_hard_counts.values,
}
frame_hard = DataFrame(temp_data,index=[1,2,3,4,5,6,7,8,9,10])

---

In [40]:
frame_soft

,数据分析,count1,数据挖掘,count2,商业分析,count3,机器学习,count4
1,数据分析,461,数据挖掘,304,商业,217,机器学习,254
2,报告,228,算法,261,数据分析,155,算法,226
3,建模,193,机器学习,237,报告,103,深度学习,128
4,数据挖掘,189,数据分析,185,咨询,86,数据挖掘,109
5,算法,136,建模,178,算法,63,数据结构,69
6,报表,135,数据库,89,数据挖掘,59,建模,68
7,数据库,132,数据结构,79,英语,51,人工智能,63
8,决策,129,评估,60,决策,50,数据分析,60
9,逻辑思维,120,逻辑思维,59,建模,49,数据库,37
10,商业,108,数据仓库,59,逻辑思维,47,逻辑思维,29


In [41]:
frame_hard

,数据分析,count1,数据挖掘,count2,商业分析,count3,机器学习,count4
1,Sql,316,Python,274,Sql,100,Python,185
2,Python,300,Hadoop,153,Python,75,Java,124
3,Excel,185,Spark,150,Spss,50,C,120
4,Sas,144,Java,140,Ppt,49,Tensorflow,90
5,Spss,112,Sql,140,Excel,49,Spark,55
6,Hive,95,Hive,106,C,44,Hadoop,47
7,Hadoop,86,C,89,Java,32,Linux,44
8,Tableau,71,Linux,60,Sas,29,Sql,36
9,Ppt,69,Shell,41,Linux,21,Hive,20
10,Spark,44,Sas,38,Hive,18,Matlab,19


---

In [42]:
from pyecharts import Bar, Line, Scatter, EffectScatter, Grid

# ---------变量
attr11 = frame_hard['数据分析'].values
v11 = frame_hard.count1.values

attr21 = frame_hard['数据挖掘'].values
v21 = frame_hard.count2.values

attr31 = frame_hard['商业分析'].values
v31 = frame_hard.count3.values

attr41 = frame_hard['机器学习'].values
v41 = frame_hard.count4.values
# ---------------

# --------------生成图表------------
bar11 = Bar("数据分析", title_pos="45%")
bar11.add("数据分析", attr11, v11,mark_point=["max"],is_legend_show=False,
         xaxis_interval=0,xaxis_rotate=30)

bar21 = Bar("数据挖掘")
bar21.add("数据挖掘",attr21,v21,mark_point=["max"],is_legend_show=False,
          xaxis_interval=0,xaxis_rotate=30)

bar31 = Bar("商业分析",title_top="50%", title_pos="45%")
bar31.add("商业分析", attr31, v31,mark_point=["max"],is_legend_show=False,
          xaxis_interval=0,xaxis_rotate=30)

bar41 = Bar("机器学习",title_top="50%")
bar41.add("机器学习", attr41, v41,mark_point=["max"],is_legend_show=False,
         xaxis_interval=0,xaxis_rotate=30)
# -----------------------------------

# ---------变量
attr12 = frame_soft['数据分析'].values
v12 = frame_soft.count1.values

attr22 = frame_soft['数据挖掘'].values
v22 = frame_soft.count2.values

attr32 = frame_soft['商业分析'].values
v32 = frame_soft.count3.values

attr42 = frame_soft['机器学习'].values
v42 = frame_soft.count4.values
# ---------------

# --------------生成图表------------
bar12 = Bar("数据分析", title_pos="45%")
bar12.add("数据分析", attr12, v12,mark_point=["max"],is_legend_show=False,
         xaxis_interval=0,xaxis_rotate=30)

bar22 = Bar("数据挖掘")
bar22.add("数据挖掘",attr22,v22,mark_point=["max"],is_legend_show=False,
          xaxis_interval=0,xaxis_rotate=30)

bar32 = Bar("商业分析",title_top="50%", title_pos="45%")
bar32.add("商业分析", attr32, v32,mark_point=["max"],is_legend_show=False,
          xaxis_interval=0,xaxis_rotate=30)

bar42 = Bar("机器学习",title_top="50%")
bar42.add("机器学习", attr42, v42,mark_point=["max"],is_legend_show=False,
         xaxis_interval=0,xaxis_rotate=30)
# -----------------------------------




grid = Grid('数据科学求职技能',height=900, width=950)
grid.add(bar11, grid_bottom="80%", grid_left="55%")
grid.add(bar21, grid_bottom="80%", grid_right="55%")

grid.add(bar32, grid_top="80%", grid_left="55%")
grid.add(bar42, grid_top="80%", grid_right="55%")

grid.add(bar12, grid_top="30%",grid_bottom="60%", grid_left="55%")
grid.add(bar22, grid_top="30%",grid_bottom="60%", grid_right="55%")

grid.add(bar31, grid_top="70%",grid_bottom="40%", grid_left="55%")
grid.add(bar41, grid_top="70%",grid_bottom="40%", grid_right="55%")
grid.render(path='./images/数据科学求职技能.png')
grid

phantomjs version: 2.1.1

Generating file ...
File saved in ./images/数据科学求职技能.png


---

# 正则提取行政区

In [43]:
data['地区'].head()

0     上海 杨浦区 五角场
1     上海 普陀区 武宁路
2    上海 静安区 共和新路
3    上海 杨浦区 复旦大学
4     上海 杨浦区 长阳路
Name: 地区, dtype: object

In [44]:
temp = data['地区'][10]

In [45]:
district = []
area = []
for item in data['地区']:
    dis_list = jieba.lcut(item)
    if dis_list[2] != ' ':
        district.append(dis_list[2])

In [46]:
dis_count = Series(district).value_counts()

In [47]:
from pyecharts import Pie

dis_count_x = dis_count.index
dis_count_y = dis_count.values
pie = Pie("各区职位数", title_pos='right')
pie.add(
    "",
    dis_count_x,
    dis_count_y,
    radius=[40, 75],
    label_text_color=None,
    is_label_show=True,
    legend_orient="vertical",
    legend_pos="left",
)
pie.render(path='./images/各区职位数.png')
pie

phantomjs version: 2.1.1

Generating file ...
File saved in ./images/各区职位数.png


---

In [48]:
frame_hard

,数据分析,count1,数据挖掘,count2,商业分析,count3,机器学习,count4
1,Sql,316,Python,274,Sql,100,Python,185
2,Python,300,Hadoop,153,Python,75,Java,124
3,Excel,185,Spark,150,Spss,50,C,120
4,Sas,144,Java,140,Ppt,49,Tensorflow,90
5,Spss,112,Sql,140,Excel,49,Spark,55
6,Hive,95,Hive,106,C,44,Hadoop,47
7,Hadoop,86,C,89,Java,32,Linux,44
8,Tableau,71,Linux,60,Sas,29,Sql,36
9,Ppt,69,Shell,41,Linux,21,Hive,20
10,Spark,44,Sas,38,Hive,18,Matlab,19


# 添加数据分析硬技能列

In [49]:
data_analysis_copy = data_analysis.copy()

In [50]:
#total_skill_list

In [51]:
# 硬技能匹配函数
total_skill_list = []
def match(words):
    tmp_list = []
    for skill in frame_hard.数据分析.values:
        if skill in words:
            tmp_list.append(skill)
    #print(tmp_list)
    #print('='*20) 
    total_skill_list.append(tmp_list)
    
# 初始化列表
for item in data_analysis_copy['职位描述'].values:
    # 匹配英文
    english_words = re.findall(pattern,item)
    # 全部转换为首字母大小，其余小写
    english_words = [item.capitalize() for item in english_words]
    english_words = list(set(english_words)) # list去重
    match(english_words)

In [52]:
skill_series = Series(total_skill_list,index=data_analysis_copy.index)
#skill_series[:10]

In [53]:
for k in frame_hard.数据分析.values:
    data_analysis_copy[k] = 0
    
for i in skill_series.index:
    for skill in skill_series[i]:
        #print(skill)
        data_analysis_copy[skill][i] = 1

## 存储到CSV

In [54]:
data_analysis_copy.to_csv('./data/data_analysis_with_skills.csv')

---

# 添加数据挖掘硬技能列

In [55]:
data_mining_copy = data_mining.copy()

In [56]:
# 硬技能匹配函数
total_skill_list = []
def match(words):
    tmp_list = []
    for skill in frame_hard.数据挖掘.values:
        if skill in words:
            tmp_list.append(skill)
    total_skill_list.append(tmp_list)
    
# ----------------------    

for item in data_mining_copy['职位描述'].values:
    # 匹配英文
    english_words = re.findall(pattern,item)
    # 全部转换为首字母大小，其余小写
    english_words = [item.capitalize() for item in english_words]
    english_words = list(set(english_words)) # list去重
    match(english_words)

In [57]:
for k in frame_hard.数据挖掘.values:
    data_mining_copy[k] = 0

In [58]:
skill_series = Series(total_skill_list,index=data_mining_copy.index)

for i in skill_series.index:
    for skill in skill_series[i]:
        data_mining_copy[skill][i] = 1

## 存储到CSV

In [59]:
data_mining_copy.to_csv('./data/data_mining_with_skills.csv')

---

# 添加机器学习硬技能列

In [60]:
machine_learning_copy = machine_learning.copy()

In [61]:
# 硬技能匹配函数
total_skill_list = []
def match(words):
    tmp_list = []
    for skill in frame_hard.机器学习.values:
        if skill in words:
            tmp_list.append(skill)
    total_skill_list.append(tmp_list)
    
# ----------------------    

for item in machine_learning_copy['职位描述'].values:
    # 匹配英文
    english_words = re.findall(pattern,item)
    # 全部转换为首字母大小，其余小写
    english_words = [item.capitalize() for item in english_words]
    english_words = list(set(english_words)) # list去重
    match(english_words)

In [62]:
for k in frame_hard.机器学习.values:
    machine_learning_copy[k] = 0
    
skill_series = Series(total_skill_list,index=machine_learning_copy.index)

for i in skill_series.index:
    for skill in skill_series[i]:
        machine_learning_copy[skill][i] = 1

## 存储到CSV


In [63]:
machine_learning_copy.to_csv('./data/machine_learning_with_skills.csv')

---

# 添加商业分析硬技能列

In [64]:
business_analysis_copy = business_analysis.copy()

In [65]:
# 硬技能匹配函数
total_skill_list = []
def match(words):
    tmp_list = []
    for skill in frame_hard.商业分析.values:
        if skill in words:
            tmp_list.append(skill)
    total_skill_list.append(tmp_list)
    
# ----------------------    

for item in business_analysis_copy['职位描述'].values:
    # 匹配英文
    english_words = re.findall(pattern,item)
    # 全部转换为首字母大小，其余小写
    english_words = [item.capitalize() for item in english_words]
    english_words = list(set(english_words)) # list去重
    match(english_words)

In [66]:
for k in frame_hard.商业分析.values:
    business_analysis_copy[k] = 0
    
skill_series = Series(total_skill_list,index=business_analysis_copy.index)

for i in skill_series.index:
    for skill in skill_series[i]:
        business_analysis_copy[skill][i] = 1

## 存储到CSV

In [67]:
business_analysis_copy.to_csv('./data/business_analysis_with_skills.csv')

---